<a href="https://colab.research.google.com/github/mrunalmania/stock-prediction/blob/main/Back_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
#!pip install TA-lib
!pip install backtesting
#!pip install tredingview-ta
#!pip install pyine==1.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=2800f6200e65703c4e0093735fee49544a5a364b6e87c16c8d2c7c15f205fb89
  Stored in directory: /root/.cache/pip/wheels/6a/76/03/7f785aaa50b9c6ec7e3fd105a62c1b2c45a034512d51e024a9
Successfully built ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173817 sha256=28086a31a9768b3e8ed6a90646377bb777052ae17a7344701f58d293baacf274
  Stored in directory: /root/.cache/pip/wheels/3f/7c/24/f8816cdb5359accfe50ebbb023baf41e98592f11528ed26ce6
Successfully built backtesting


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10622089995953601078
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9429869177065925531
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
import yfinance as yf
import ta
import pandas as pd
#import pandas_ta as p_ta
from backtesting import Backtest, Strategy
from backtesting. lib import crossover

/usr/local/lib/python3.9/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
class SMAcross(Strategy):
    n1 = 50
    n2 = 100

    def init(self):
        close = self.data.Close
        self.smal = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)

    def next(self):
        if crossover(self.smal, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.smal):
            self.sell()

In [ ]:
class EMAcross(Strategy):
    n1 = 50
    n2 = 200

    def init(self):
        close = self.data.Close
        self.emal = self.I(ta.trend.ema_indicator, pd.Series(close), self.n1)
        self.ema2 = self.I(ta.trend.ema_indicator, pd.Series(close), self.n2)

    def next(self):
        if crossover(self.emal, self.ema2):
            self.buy()
        elif crossover(self.ema2, self.emal):
            self.sell()

In [ ]:
df = yf.download('^GSPC', start='2000-01-01' )
price_diff = df['Close'].diff(periods=1)
pct_change = df['Close'].pct_change(periods=1) * 100
df['Percentage Change'] = pct_change
#df = df[pct_change>5]

[*********************100%***********************]  1 of 1 completed


In [ ]:
#a_df = pd.read_csv('All_Subtracted_data.csv')
#a_df['Date'] = pd.to_datetime(a_df['Date'])

In [ ]:
#df = pd.merge(a_df, df, on = "Date")

In [ ]:
bt = Backtest(df, EMAcross, cash=100000, commission=0.002, exclusive_orders=True)

In [ ]:
output = bt. run ( )
output

Start                     2000-01-03 00:00:00
End                       2023-04-06 00:00:00
Duration                   8494 days 00:00:00
Exposure Time [%]                   85.392107
Equity Final [$]                186318.104767
Equity Peak [$]                 267037.658677
Return [%]                          86.318105
Buy & Hold Return [%]              182.089313
Return (Ann.) [%]                    2.715454
Volatility (Ann.) [%]               16.490093
Sharpe Ratio                         0.164672
Sortino Ratio                        0.234642
Calmar Ratio                         0.050041
Max. Drawdown [%]                  -54.264662
Avg. Drawdown [%]                   -2.579424
Max. Drawdown Duration     2877 days 00:00:00
Avg. Drawdown Duration       59 days 00:00:00
# Trades                                   22
Win Rate [%]                        36.363636
Best Trade [%]                      49.320771
Worst Trade [%]                    -25.330462
Avg. Trade [%]                    

In [ ]:
optim = bt.optimize(n1 = range(50,200,10),
                    n2 = range(50,200,10),
                    constraint = lambda x: x.n2-x.n1 > 20,
                    maximize= 'Return [%]')
bt.plot()

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Row(id='1449', ...)